In [50]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlayWrightTimeout
import time


In [52]:
SEASONS = list(range(2019,2024)) #constant that captures seasons 2019-2020 to 2023-2024



In [54]:
DATA_DIR = "data0" # directory storing all the data

STANDINGS_DIR = os.path.join(DATA_DIR, "standings") # a directory that shows the standings that are scrapped as standings from the html

SCORES_DIR = os.path.join(DATA_DIR, "scores") # directory to store all the scores

In [56]:
async def get_html (url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1): # retries are performed by interval of 1 
        time.sleep(sleep* i) # time between retries is done by mutilple of 5 secs

        try:
            async with async_playwright() as p: 
                browser = await p.chromium.launch() # launches open source version of chrome
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlayWrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [58]:
async def scrape_season (season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    if html is None:
        print(f"Failed to retrieve HTML for season{season}")
        return
    
    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
            save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
            if os.path.exists(save_path):
                continue
            
            html = await get_html(url,"#all_schedule")
            if html is None: 
                print(f"Failed to retrieve HTML for {url}")
                continue
              
            with open(save_path, "w+") as f: 
                f.write(html)

In [60]:
for season in SEASONS:
    await scrape_season(season)


2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games-may.html
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games-may.html
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2020_games.html
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Bas